## Recommendation systems  :
are among the most popular applications of data science. They are used to predict the Rating or Preference that a user would give to an item.

Almost every major company has applied them in some form or the other: Amazon uses it to suggest products to customers, YouTube uses it to decide which video to play next on auto play, and Facebook uses it to recommend pages to like and people to follow.

# 1. import library and data



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
books = pd.read_csv('BX_Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']


<ipython-input-1-83187dcccc92>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('BX_Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")


In [2]:
print(books['bookTitle'].head(50))

0                                   Classical Mythology
1                                          Clara Callan
2                                  Decision in Normandy
3     Flu: The Story of the Great Influenza Pandemic...
4                                The Mummies of Urumchi
5                                The Kitchen God's Wife
6     What If?: The World's Foremost Military Histor...
7                                       PLEADING GUILTY
8     Under the Black Flag: The Romance and the Real...
9               Where You'll Find Me: And Other Stories
10                          Nights Below Station Street
11    Hitler's Secret Bankers: The Myth of Swiss Neu...
12                                   The Middle Stories
13                                             Jane Doe
14    A Second Chicken Soup for the Woman's Soul (Ch...
15         The Witchfinder (Amos Walker Mystery Series)
16    More Cunning Than Man: A Social History of Rat...
17                        Goodbye to the Butterm

In [ ]:
users = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']

<ipython-input-32-16e8c0f25019>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")


In [ ]:
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

<ipython-input-33-ba25cdc8e13b>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")


In [ ]:
print(ratings.shape)
print(list(ratings.columns))
print(users.shape)
print(list(users.columns))

(1149780, 3)
['userID', 'ISBN', 'bookRating']
(278858, 3)
['userID', 'Location', 'Age']


### To ensure statistical significance, users with less than 200 ratings, and books with less than 100 ratings are excluded.

In [ ]:
counts1 = ratings['userID'].value_counts()
ratings = ratings[ratings['userID'].isin(counts1[counts1 >= 200].index)]
counts = ratings['bookRating'].value_counts()
ratings = ratings[ratings['bookRating'].isin(counts[counts >= 100].index)]

## Collaborative Filtering Using k-Nearest Neighbors (kNN)
kNN is a machine learning algorithm to find clusters of similar users based on common book ratings, and make predictions using the average rating of top-k nearest neighbors.

For example, we first present ratings in a matrix with the matrix having one row for each item (book) and one column for each user.

In [ ]:
combine_book_rating = pd.merge(ratings, books, on='ISBN')
columns = ['yearOfPublication', 'publisher', 'bookAuthor', 'imageUrlS', 'imageUrlM', 'imageUrlL']
combine_book_rating = combine_book_rating.drop(columns, axis=1)
print(combine_book_rating.head())

   userID        ISBN  bookRating  \
0  277427  002542730X          10   
1    3363  002542730X           0   
2   11676  002542730X           6   
3   12538  002542730X          10   
4   13552  002542730X           0   

                                           bookTitle  
0  Politically Correct Bedtime Stories: Modern Ta...  
1  Politically Correct Bedtime Stories: Modern Ta...  
2  Politically Correct Bedtime Stories: Modern Ta...  
3  Politically Correct Bedtime Stories: Modern Ta...  
4  Politically Correct Bedtime Stories: Modern Ta...  


## Collaborative Filtering Using k-Nearest Neighbors (kNN)
kNN is a machine learning algorithm to find clusters of similar users based on common book ratings, and make predictions using the average rating of top-k nearest neighbors.

For example, we first present ratings in a matrix with the matrix having one row for each item (book) and one column for each user.

In [ ]:
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['bookTitle'])

book_ratingCount = (combine_book_rating.
     groupby(by = ['bookTitle'])['bookRating'].
     count().
     reset_index().
     rename(columns = {'bookRating': 'totalRatingCount'})
     [['bookTitle', 'totalRatingCount']]
    )
print(book_ratingCount.head())

                                           bookTitle  totalRatingCount
0   A Light in the Storm: The Civil War Diary of ...                 2
1                              Always Have Popsicles                 1
2               Apple Magic (The Collector's series)                 1
3   Beyond IBM: Leadership Marketing and Finance ...                 1
4   Clifford Visita El Hospital (Clifford El Gran...                 1


### Now we will combine the rating data with the total rating count data, this gives us exactly what we need to find out which books are popular and filter out lesser-known books.

In [ ]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'bookTitle', right_on = 'bookTitle', how = 'left')
print(rating_with_totalRatingCount.head())

pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(book_ratingCount['totalRatingCount'].describe())

   userID        ISBN  bookRating  \
0  277427  002542730X          10   
1    3363  002542730X           0   
2   11676  002542730X           6   
3   12538  002542730X          10   
4   13552  002542730X           0   

                                           bookTitle  totalRatingCount  
0  Politically Correct Bedtime Stories: Modern Ta...                82  
1  Politically Correct Bedtime Stories: Modern Ta...                82  
2  Politically Correct Bedtime Stories: Modern Ta...                82  
3  Politically Correct Bedtime Stories: Modern Ta...                82  
4  Politically Correct Bedtime Stories: Modern Ta...                82  
count   160587.000
mean         3.044
std          7.428
min          1.000
25%          1.000
50%          1.000
75%          2.000
max        365.000
Name: totalRatingCount, dtype: float64


In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(book_ratingCount['totalRatingCount'].describe())

count   160587.000
mean         3.044
std          7.428
min          1.000
25%          1.000
50%          1.000
75%          2.000
max        365.000
Name: totalRatingCount, dtype: float64


In [ ]:
print(book_ratingCount['totalRatingCount'].quantile(np.arange(.9, 1, .01)))

0.900    5.000
0.910    6.000
0.920    7.000
0.930    7.000
0.940    8.000
0.950   10.000
0.960   11.000
0.970   14.000
0.980   19.000
0.990   31.000
Name: totalRatingCount, dtype: float64


In [ ]:
popularity_threshold = 50
rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
print(rating_popular_book.head())

   userID        ISBN  bookRating  \
0  277427  002542730X          10   
1    3363  002542730X           0   
2   11676  002542730X           6   
3   12538  002542730X          10   
4   13552  002542730X           0   

                                           bookTitle  totalRatingCount  
0  Politically Correct Bedtime Stories: Modern Ta...                82  
1  Politically Correct Bedtime Stories: Modern Ta...                82  
2  Politically Correct Bedtime Stories: Modern Ta...                82  
3  Politically Correct Bedtime Stories: Modern Ta...                82  
4  Politically Correct Bedtime Stories: Modern Ta...                82  


# Filter to users in US and Canada only

In [ ]:
combined = rating_popular_book.merge(users, left_on = 'userID', right_on = 'userID', how = 'left')

us_canada_user_rating = combined[combined['Location'].str.contains("usa|canada")]
us_canada_user_rating=us_canada_user_rating.drop('Age', axis=1)
print(us_canada_user_rating.head())

   userID        ISBN  bookRating  \
0  277427  002542730X          10   
1    3363  002542730X           0   
3   12538  002542730X          10   
4   13552  002542730X           0   
5   16795  002542730X           0   

                                           bookTitle  totalRatingCount  \
0  Politically Correct Bedtime Stories: Modern Ta...                82   
1  Politically Correct Bedtime Stories: Modern Ta...                82   
3  Politically Correct Bedtime Stories: Modern Ta...                82   
4  Politically Correct Bedtime Stories: Modern Ta...                82   
5  Politically Correct Bedtime Stories: Modern Ta...                82   

                        Location  
0          gilbert, arizona, usa  
1      knoxville, tennessee, usa  
3          byron, minnesota, usa  
4        cordova, tennessee, usa  
5  mechanicsville, maryland, usa  


## Implementing kNN
We convert our table to a 2D matrix, and fill the missing values with zeros (since we will calculate distances between rating vectors).

We then transform the values(ratings) of the matrix dataframe into a scipy sparse matrix for more efficient calculations.

In [ ]:
from scipy.sparse import csr_matrix
us_canada_user_rating = us_canada_user_rating.drop_duplicates(['userID', 'bookTitle'])
us_canada_user_rating_pivot = us_canada_user_rating.pivot(index = 'bookTitle', columns = 'userID', values = 'bookRating').fillna(0)
us_canada_user_rating_matrix = csr_matrix(us_canada_user_rating_pivot.values)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(us_canada_user_rating_matrix)
print(model_knn)

NearestNeighbors(algorithm='brute', metric='cosine')


In [ ]:
query_index = np.random.choice(us_canada_user_rating_pivot.shape[0])
#print(query_index)
#print(us_canada_user_rating_pivot.iloc[query_index,:].values.reshape(1,-1))
distances, indices = model_knn.kneighbors(us_canada_user_rating_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
us_canada_user_rating_pivot.index[query_index]

'The Tommyknockers'

In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(us_canada_user_rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, us_canada_user_rating_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for The Tommyknockers:

1: Four Past Midnight, with distance of 0.42040790615695145:
2: The Dark Half, with distance of 0.44504614508045803:
3: The Regulators, with distance of 0.4852679647352577:
4: The Dead Zone, with distance of 0.4854464982734821:
5: Gerald's Game, with distance of 0.49665928093393097:


# Intents


In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

with open('my_intents.json') as file:
    data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_3 (Dense)             (None, 128)               2176      
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 7)                 455       
                                                                 
Total params: 26887 (105.03 KB)
Trainable params: 26887 (105.03 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
epochs = 1000
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/1000
1/1 [==============================] - 1s 898ms/step - loss: 1.9474 - accuracy: 0.2692
Epoch 2/1000
1/1 [==============================] - 0s 15ms/step - loss: 1.9443 - accuracy: 0.2692
Epoch 3/1000
1/1 [==============================] - 0s 12ms/step - loss: 1.9416 - accuracy: 0.2692
Epoch 4/1000
1/1 [==============================] - 0s 10ms/step - loss: 1.9391 - accuracy: 0.2692
Epoch 5/1000
1/1 [==============================] - 0s 12ms/step - loss: 1.9366 - accuracy: 0.2692
Epoch 6/1000
1/1 [==============================] - 0s 12ms/step - loss: 1.9343 - accuracy: 0.2692
Epoch 7/1000
1/1 [==============================] - 0s 13ms/step - loss: 1.9319 - accuracy: 0.2692
Epoch 8/1000
1/1 [==============================] - 0s 12ms/step - loss: 1.9294 - accuracy: 0.2692
Epoch 9/1000
1/1 [==============================] - 0s 12ms/step - loss: 1.9268 - accuracy: 0.2692
Epoch 10/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.9241 - accuracy: 0.2692
Epoch 11

In [ ]:
%%capture
!pip install joblib

In [ ]:
import joblib
joblib.dump(model, 'chat_model.pkl')
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
%%capture
!pip install colorama

In [ ]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
import colorama
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

# load trained model
model = keras.models.load_model('chat_model')

# load tokenizer object
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# load label encoder object
with open('label_encoder.pickle', 'rb') as enc:
    lbl_encoder = pickle.load(enc)

# parameters
max_len = 20

with open("my_intents.json") as file:
    data = json.load(file)

def chat(inp):
    # while True:
    #     print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
    #     inp = input()
    #     if inp.lower() == "quit":
    #         break

    result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                          truncating='post', maxlen=max_len), verbose = False)
    tag = lbl_encoder.inverse_transform([np.argmax(result)])

    for i in data['intents']:
        if i['tag'] == tag:
          return  np.random.choice(i['responses'])
            #print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

# print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
# chat()

In [ ]:
def recommendation(sss):
    result = ""  # Variable to store the recommendations

    query_index = us_canada_user_rating_pivot.index.get_loc(sss)

    # Calculate distances and indices using the selected book
    distances, indices = model_knn.kneighbors(us_canada_user_rating_pivot.iloc[query_index, :].values.reshape(1, -1), n_neighbors=6)

    # Append the selected book name to the result
    result += f"Selected Book: {us_canada_user_rating_pivot.index[query_index]}\n"

    # Append the recommendations to the result
    for i in range(len(distances.flatten())):
        if i == 0:
            result += 'Recommendations for {0}:\n'.format(us_canada_user_rating_pivot.index[query_index])
        else:
            result += '{0}: {1}, with distance of {2}:\n'.format(i, us_canada_user_rating_pivot.index[indices.flatten()[i]], distances.flatten()[i])

    return result


# Summarizer

In [ ]:
%%capture
!pip install transformers

In [ ]:
import json
with open('books_to_read_actual.json') as json_file:
    books_data = json.load(json_file)

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

def bart_abstractive_summarizer(chapter_content, max_length=200):
    model_name = "facebook/bart-large-cnn"
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    inputs = tokenizer.encode("summarize: " + chapter_content, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def read_chapter_with_summarization(book_data, chapter_index):
    chapter_key = f"Chapter-{chapter_index}"
    chapter_content = book_data.get(chapter_key, "")
    if chapter_content:
        summary = bart_abstractive_summarizer(chapter_content)

        print("\nSummary of the chapter:")
        print(summary)

        return "Chapter reading completed."
    else:
        return f"Chapter {chapter_index} does not have content available."

In [ ]:
def get_book_and_chapter_for_summary(book_name):
    # book_name = 'rich dad poor dad' #input("Enter the name of the book: ")

    selected_book = None
    for book in books_data:
        if book["Title"].lower() == book_name.lower():
            selected_book = book
            break

    if selected_book:
        print("\nContents of the book:")
        for i, chapter in enumerate(selected_book["Contents"], start=1):
            print(f"{i}. {chapter}")

        chapter_index = 1 #int(input("\nEnter the chapter number to read: "))
        result = read_chapter_with_summarization(selected_book, chapter_index)
        return result
    else:
        return f"Book '{book_name}' not found in the library."


In [ ]:
print("Start messaging with the bot (type /stop to quit)!")
while True:
    user_input = input("You: ")

    if user_input.lower().strip() == "/stop":
        break

    if "recommendation" in user_input.lower() or "recommend" in user_input.lower():
        # Extract book name from text between single quotes
        book_name_start = user_input.find("'") + 1
        book_name_end = user_input.find("'", book_name_start)

        if book_name_start != -1 and book_name_end != -1:
            book_name = user_input[book_name_start:book_name_end]
            response = recommendation(book_name)
        else:
            response = "I'm sorry, I couldn't find the book name in your request."
    elif "summary" in user_input.lower() or "summarize" in user_input.lower():
        book_name_start = user_input.find("'") + 1
        book_name_end = user_input.find("'", book_name_start)

        if book_name_start != -1 and book_name_end != -1:
            book_name = user_input[book_name_start:book_name_end]
            response = get_book_and_chapter_for_summary(book_name)
    else:

        response = chat(user_input)

    print("Bot:", response)
    print("------------------------------------------------------------------")









Start messaging with the bot (type /stop to quit)!
You: Hello
Bot: Hello, thanks for asking
------------------------------------------------------------------
You: what can you do for me?
Bot: I am like your virtual assistant, you can ask me any questions. If I know I will answer
------------------------------------------------------------------
You: Where can I find story books?
Bot: Story books section is on the 4th row from entrance
------------------------------------------------------------------
You: Can you tell me the cost for harry potter azkaban book?
Bot: You can buy it for 280tk
------------------------------------------------------------------
You: thanks
Bot: Any time!
------------------------------------------------------------------
You: Bye
Bot: Have a nice day
------------------------------------------------------------------
You: /stop
